# Install and import Dependencies

In [ ]:
! pip install tensorflow==2.4.1 tensorflow-gpu==2.4.1 opencv-python matplotlib

In [1]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import tensorflow as tf
import os
import math
import random

In [2]:

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)

In [3]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Load Model 

In [4]:
model_path = '1.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Make Detections

In [91]:
# People running - aspect ratio = 0.5625 , resize chosen - 192 , 320
# cap = cv.VideoCapture("people_running.mp4")
cap = cv.VideoCapture("football_2.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    #### Input and resizing
    image = frame.copy()
    image = tf.image.resize_with_pad(np.expand_dims(image,axis=0),192,320)
    image = tf.cast(image,dtype=tf.uint8)

    # ### set output and input
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    is_dynamic_shape_model = input_details[0]['shape_signature'][2] == -1 # if true, means it is dynamic model
    if is_dynamic_shape_model:
        input_tensor_index = input_details[0]['index']
        input_shape = image.shape
        interpreter.resize_tensor_input(
            input_tensor_index, input_shape, strict=True)
    interpreter.allocate_tensors()

    # #make Predictions
    interpreter.set_tensor(input_details[0]['index'], image.numpy())
    interpreter.invoke()
    results = interpreter.get_tensor(output_details[0]['index'])

    keypoints_with_scores = results[:,:,:51].reshape((6,17,3))
    draw_keypoints_connections(frame,keypoints_with_scores,EDGES,0.4)
    draw_keypoints(frame,keypoints_with_scores,0.4)

    cv.imshow("Frame",frame)
   
    if cv.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()

# Testing code

In [ ]:
img = np.squeeze(tf.image.resize_with_pad(np.expand_dims(frame,axis=0),256,256))
plt.imshow(cv.cvtColor(img,cv.COLOR_BGR2RGB).astype(np.int64))

In [12]:
# keypoints_with_scores
is_dynamic_shape_model = input_details[0]['shape_signature'][2] == -1

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([1, 1, 1, 3]),
  'shape_signature': array([ 1, -1, -1,  3]),
  'dtype': numpy.uint8,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [13]:
is_dynamic_shape_model = input_details[0]['shape_signature'][2] == -1
if is_dynamic_shape_model:
    input_tensor_index = input_details[0]['index']
    input_shape = frame.shape
    interpreter.resize_tensor_input(
        input_tensor_index, input_shape, strict=True)

In [ ]:
keypoints_with_scores

In [27]:
np.multiply(keypoints_with_scores,[480,640,1]).shape

(6, 17, 3)

In [48]:
# array = np.zeros((3,1,1))
# print(array.shape)
# array = np.squeeze(array)
# print(array.shape)

(3, 1, 1)
(3,)


# Draw Keypoints

In [30]:
def draw_keypoints(frame,keypoints,confidence):
    y,x,c = frame.shape
    shaped_keypoints = np.squeeze(np.multiply(keypoints,[y,x,1]))
    for person_keypoint in shaped_keypoints:
        for kp in person_keypoint:
            ky,kx,kc = kp
            if kc > confidence:
                cv.circle(frame,(int(kx),int(ky)),4,(0,255,0),-1)
        

# Draw Edges or connections

In [52]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [50]:
import random
import math
def draw_keypoints_connections(frame,keypoints,edges,confidence_threshold):
    y,x,c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints,[y,x,1]))
    
    for points, color in EDGES.items():
        p1,p2 = points
        for person in shaped:
            y1,x1,c1 = person[p1]
            y2,x2,c2 = person[p2]
            if (c1 > confidence_threshold) & (c2 > confidence_threshold):
                cv.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (math.floor(random.random() * 100),math.floor(random.random() * 160),math.floor(random.random() * 255)), 2)